<a href="https://colab.research.google.com/github/vijayrgopu/RESTApi/blob/master/socotra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Requests and prettyprint libraries

In [0]:
#Importing requests lib for REST API
import requests
import pprint

# Get Authorization Token using credentials provided

In [0]:
#Using credentials provided get Auth Token
def get_auth_token():
  auth_url = 'https://api.sandbox.socotra.com/account/authenticate'
  host_name = 'vgopu-sata-configeditor.co.sandbox.socotra.com'
  
  response = requests.post(
          auth_url,
          json={'hostName': host_name,
                'username':'alice.lee',
                'password':'socotra'})

  #from the response dictionary get the token using the key authorizationToken
  authToken = response.json()['authorizationToken']
  return authToken

# Create a Session and Update Authorization Token

In [0]:
#update session with received Token
def get_session(authToken):
  #creating Headers
  headers = {'Authorization': authToken}
  session = requests.Session()
  session.headers.update(headers)
  return session

# Create a Policy Holder

In [0]:
#Creating a policy holder
def create_policyholder(session):
  create_policyholder_url = 'https://api.sandbox.socotra.com/policyholder/create'
  policyholder_values = {'first_name': 'Vijaya',
                         'last_name': 'Gopu',
                         'date_of_birth': '1987-10-03',
                         'policyholder_id': 'placeholder'}
  
  response = session.post(
        create_policyholder_url,
        json={'values': policyholder_values})
  policyholder = response.json()
  return policyholder

# Create Insured Vehicle or Exposure

In [0]:
#Creating Exposure
def create_exposure():
  exposure_values = {'vehicle_type': 'Car',
                     'make': 'Acura',
                     'model': 'TSX',
                     'year': '2017',
                     'vehicle_value': '35000',
                     'license_plate': '6NAX604',
                     'vehicle_color': 'Blue'#Include vehicle color because we added vehicle_color as new property while configuring
                    }

  exposure = {'exposureName': 'vehicle',
              'fieldValues': exposure_values,
              'perils': [{'name': 'bodily_injury'},
                         {'name': 'third_party_liability'}]
             }
  return exposure

# Create a Policy for Exposure

In [0]:
#Create policy
def create_policy(session,policyholder,exposure):
  create_policy_url = 'https://api.sandbox.socotra.com/policy'
  policyholder_locator = policyholder['locator']
  #create policy channel
  policy_values = {'channel': 'Direct'}

  #policy dictionary
  policy = { 'policyholderLocator': policyholder_locator,
             'productName': 'simple_auto',
             'fieldValues': policy_values,
             'exposures': [exposure]
           }

  response = session.post(create_policy_url,json=policy)
  #pp = pprint.PrettyPrinter(indent=1)
  #pp.pprint(response.json())
  policy = response.json()
  return policy

# Get the Invoice Information using policyID or policy locator

In [0]:
#get policy data from API
def get_invoice(session, policy):
  policy_locator = policy['locator']
  invoice_response = session.get(
          'https://api.sandbox.socotra.com/policy/' + policy_locator)
  display_id = invoice_response.json()['displayId']
  num_of_invoices = len(invoice_response.json()['invoices'])

  print ('Policy id: %s' % display_id)
  print ('Number of invoices on Policy: %s' % num_of_invoices)

# Controller Program to Invoke functions and return Invoice Info

In [8]:
if __name__ == "__main__":
  authToken = get_auth_token()
  session = get_session(authToken)
  policyholder = create_policyholder(session)
  exposure = create_exposure()
  policy = create_policy(session,policyholder,exposure)
  get_invoice(session, policy)

Policy id: 100000061
Number of invoices on Policy: 1
